In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format='retina'

In [2]:
store = pd.read_csv('data/store.csv', index_col = 0)
train = pd.read_csv('data/train.csv', index_col = 0)
test = pd.read_csv('data/test.csv', index_col = 0)

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
//anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
from help import BOW, data_preprocess

store_pre, Bow_Matrix = BOW(Data=store)
X1 = data_preprocess(train.drop(['Sales','Customers'], axis=1),
                       extend_data=store_pre, BOW_key = Bow_Matrix.columns)
X2 = data_preprocess(test.drop('Id', axis=1),
                            extend_data=store_pre, BOW_key = Bow_Matrix.columns)
X2[['StateHoliday_b', 'StateHoliday_c']] = pd.DataFrame(np.zeros((len(test),2)), 
                                                               columns=['StateHoliday_b', 'StateHoliday_c'], 
                                                               dtype='uint8')
y1 = train['Sales']
del store, test, train, store_pre, Bow_Matrix